In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train=pd.read_csv('train.csv')
sd=pd.read_csv('service_delivery_site_data.csv')
submission=pd.read_csv('SampleSubmission.csv')
monthly_cases = pd.read_csv('monthly_cases.csv')
products = pd.read_csv('product.csv')

In [3]:
print(train.shape,submission.shape,sd.shape, monthly_cases.shape,products.shape)

(35753, 14) (3089, 2) (156, 6) (678, 28) (11, 3)


In [4]:
products.head()

,product_code,product_type,product_name
0,AS17005,Female Condom,PRESERVATIF FEMININ UNITE
1,AS46000,Male Condom,PRESERVATIF MASCULIN BTE/144 BOITE
2,AS27139,Emergency Contraceptive (Pill),LEVONORGESTREL 1.5 MG CP BOITE
3,AS27137,Contraceptive Implant,JADELLE (IMPLANT + TROCARD) 150 µG BTE/10 BOITE
4,AS27138,Contraceptive Implant,IMPLANON IMPLANT 68 MG BTE/1 BOITE


In [5]:
train.head(3)

,year,month,region,district,site_code,product_code,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,75,0,21,-54,0,18,0,100.0
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3,0,3,0,0,2,0,10.0
2,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27000,0,75,22,0,53,9,0,0.0


In [6]:
monthly_cases.head(3)

,district,month,year,implant_women_old,implant_women_new,injection2_women_old,injection2_women_new,injection3_women_old,injection3_women_new,pill_women_old,pill_women_new,iud_women_old,iud_women_new,iud_number_dispensed,implant_number_dispensed,injection2_number_dispensed,injection3_number_dispensed,pill_number_dispensed,iud_number_received,implant_number_received,injection2_number_received,injection3_number_received,pill_number_received,iud_stock_end,implant_stock_end,injection2_stock_end,injection3_stock_end,pill_stock_end
0,ABENGOUROU,1,2019,478.0,91.0,103.0,17.0,1278,296,195.0,34.0,NaN,NaN,NaN,75.0,44.0,1082.0,2542.0,NaN,319.0,33.0,1244.0,2846.0,NaN,422.0,NaN,1501.0,3466.0
1,ABENGOUROU,2,2019,340.0,81.0,126.0,10.0,1237,253,187.0,19.0,9.0,1.0,1.0,66.0,38.0,813.0,557.0,20.0,255.0,17.0,1174.0,2608.0,NaN,394.0,NaN,1535.0,3327.0
2,ABENGOUROU,3,2019,367.0,59.0,14.0,7.0,1300,224,147.0,26.0,NaN,5.0,5.0,47.0,42.0,819.0,665.0,50.0,214.0,50.0,1050.0,2148.0,NaN,340.0,NaN,1187.0,3636.0


In [7]:
sd.head(3)

,site_code,site_type,site_region,site_district,site_latitude,site_longitude
0,C1399,Health Center,ABIDJAN 2,COCODY-BINGERVILLE,5.352721,-3.940210
1,C4001,Hospital,INDENIE-DJUABLIN,ABENGOUROU,6.730937,-3.483863
2,C1004,Hospital,AGNEBY-TIASSA-ME,AGBOVILLE,5.928340,-4.211450


In [8]:
train.drop(columns={'stock_initial', 'stock_received','stock_adjustment', 
                    'stock_end','average_monthly_consumption',
                    'stock_stockout_days', 'stock_ordered'},inplace=True)

In [9]:
test=pd.DataFrame(submission.ID.str.split('X',3).tolist(), columns = ['year','month','site_code','product_code'])


for col in test.columns:
    test[col] = test[col].str.strip()
column=['year','month'] 
for col in column:
    test[col]=test[col].astype(int)
test=pd.merge(test,sd,on='site_code',how='left')
test.rename(columns={'site_region':'region','site_district':'district'},inplace=True)
test=test[['year', 'month', 'region','district','site_code', 'product_code']]

In [10]:
test.head(3)

,year,month,region,district,site_code,product_code
0,2019,7,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS21126
1,2019,7,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134
2,2019,7,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132


In [11]:
train.head(2).append(test.head(2))

,year,month,region,district,site_code,product_code,stock_distributed
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,21.0
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3.0
0,2019,7,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS21126,NaN
1,2019,7,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,NaN


In [12]:
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])

In [13]:
df.head()

,year,month,region,district,site_code,product_code,stock_distributed,train_or_test
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,21.0,train
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3.0,train
2,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27000,22.0,train
3,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27137,0.0,train
4,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27138,2.0,train


In [14]:
train.head()

,year,month,region,district,site_code,product_code,stock_distributed,train_or_test
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,21,train
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3,train
2,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27000,22,train
3,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27137,0,train
4,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27138,2,train


In [15]:
sd_ls = [col for col in sd.site_code.drop_duplicates()]
df_ls = [col for col in test.site_code.drop_duplicates()]

intersects = np.intersect1d(sd_ls, df_ls)

In [16]:
sd.sort_values(by=['site_code'])

,site_code,site_type,site_region,site_district,site_latitude,site_longitude
2,C1004,Hospital,AGNEBY-TIASSA-ME,AGBOVILLE,5.928340,-4.211450
19,C1007,University Hospital/National Institute,ABIDJAN 1-GRANDS PONTS,YOPOUGON-OUEST-SONGON,5.358775,-4.089376
119,C1008,Hospital,SUD-COMOE,ABOISSO,5.414500,-3.031500
41,C1009,Health Center,AGNEBY-TIASSA-ME,TIASSALE,5.862030,-4.764830
21,C1010,Health Center,ABIDJAN 2,ABOBO-EST,5.410389,-4.019850
22,C1011,Health Center,ABIDJAN 2,ABOBO-EST,5.435494,-4.020458
23,C1014,Health Center,ABIDJAN 1-GRANDS PONTS,ADJAME-PLATEAU-ATTECOUBE,5.361543,-4.032595
24,C1015,Health Center,ABIDJAN 2,KOUMASSI-PORT BOUET-VRIDI,5.311644,-3.945893
25,C1017,Health Center,ABIDJAN 1-GRANDS PONTS,YOPOUGON-EST,5.364084,-4.066931
26,C1018,Health Center,ABIDJAN 2,COCODY-BINGERVILLE,5.399213,-3.989978


In [17]:
df.sort_values(by=['site_code'])

,year,month,region,district,site_code,product_code,stock_distributed,train_or_test
7527,2018,5,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27138,0.0,train
7569,2018,12,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27139,0.0,train
7568,2018,12,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27000,60.0,train
7567,2018,12,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27132,9.0,train
7566,2018,12,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27134,60.0,train
...,...,...,...,...,...,...,...,...
3083,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27132,NaN,test
3084,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27000,NaN,test
3085,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27139,NaN,test
3086,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27137,NaN,test


In [18]:
df['site_type'] = 0
df['lat'] = 0
df['long'] = 0

df.set_index('site_code', inplace=True)
sd.set_index('site_code', inplace = True)

for i,v in enumerate(df.index):

  for j in sd.index:

    if v == j:
      df.loc[v, 'site_type'] = sd.loc[v, 'site_type']
      df.loc[v, 'lat'] = sd.loc[v,'site_latitude']
      df.loc[v, 'long'] = sd.loc[v, 'site_longitude']


In [19]:
df.reset_index( inplace=True)
train.reset_index(inplace =True)

In [20]:
df.head()

,site_code,year,month,region,district,product_code,stock_distributed,train_or_test,site_type,lat,long
0,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27134,21.0,train,Hospital,6.730937,-3.483863
1,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27132,3.0,train,Hospital,6.730937,-3.483863
2,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27000,22.0,train,Hospital,6.730937,-3.483863
3,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27137,0.0,train,Hospital,6.730937,-3.483863
4,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27138,2.0,train,Hospital,6.730937,-3.483863


In [21]:
df = df.reset_index()

In [22]:
df.head()

,index,site_code,year,month,region,district,product_code,stock_distributed,train_or_test,site_type,lat,long
0,0,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27134,21.0,train,Hospital,6.730937,-3.483863
1,1,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27132,3.0,train,Hospital,6.730937,-3.483863
2,2,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27000,22.0,train,Hospital,6.730937,-3.483863
3,3,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27137,0.0,train,Hospital,6.730937,-3.483863
4,4,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27138,2.0,train,Hospital,6.730937,-3.483863


In [23]:
df.to_csv('DataFrame.csv')

In [24]:
data =pd.read_csv('DataFrame.csv')

In [25]:
data.drop(columns=['Unnamed: 0'], axis  = 1, inplace=True)

In [26]:
data = data.drop('index', axis='columns')
data.head(3)

,site_code,year,month,region,district,product_code,stock_distributed,train_or_test,site_type,lat,long
0,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27134,21.0,train,Hospital,6.730937,-3.483863
1,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27132,3.0,train,Hospital,6.730937,-3.483863
2,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27000,22.0,train,Hospital,6.730937,-3.483863


In [27]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_features = ['region','district','site_code','product_code','site_type']

for col in ['region','district','site_code','product_code','site_type']:
  data[col]=  data[col].astype('str')
  data[col]= le.fit_transform(data[col])

In [28]:
data.shape

(38842, 11)

In [29]:
train=data.loc[data.train_or_test.isin(['train'])]
test=data.loc[data.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
print(len(test),len(submission))

3089 3089


In [31]:
train.month.isna().sum()

0

In [32]:
X=train.drop(columns={'stock_distributed'})
y=train.loc[:,['stock_distributed']]
del test['stock_distributed']

In [33]:
print(X.shape, y.shape)

(35753, 9) (35753, 1)


In [34]:
X.head(3)

,site_code,year,month,region,district,product_code,site_type,lat,long
0,119,2019,1,11,0,5,1,6.730937,-3.483863
1,119,2019,1,11,0,3,1,6.730937,-3.483863
2,119,2019,1,11,0,2,1,6.730937,-3.483863


In [ ]:
test.head()

In [ ]:
!pip install mlxtend

In [ ]:
!pip install geopy

In [35]:
# calculate distance between the coordinates
from geopy.distance import vincenty


X['distance'] = X.apply(lambda x : vincenty(x['lat'], x['long']), axis = 1)
test['distance'] = test.apply(lambda x: vincenty(x['lat'], x['long']), axis=1)

/usr/local/lib/python3.6/dist-packages/geopy/point.py:180: UserWarning: A single number has been passed to the Point constructor. This is probably a mistake, because constructing a Point with just a latitude seems senseless. If this is exactly what was meant, then pass the zero longitude explicitly to get rid of this warning.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
X.drop(columns=['lat', 'long'], axis = 1, inplace =True)
test.drop(columns=['lat','long'], axis = 1, inplace =True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [37]:
X.distance = X.distance.apply(lambda x : x.kilometers)

In [38]:
test.distance = test.distance.apply(lambda x : x.kilometers)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [39]:
# Standardizing #
from sklearn.preprocessing import StandardScaler,MinMaxScaler


scaler = StandardScaler()
scaler2 = MinMaxScaler(feature_range=(0,1))


X[['distance']] = scaler.fit_transform(X[['distance']])
test[['distance']] = scaler.fit_transform(test[['distance']])

X[['month']] = scaler.fit_transform(X[['month']])
X[['year']] = scaler.fit_transform(X[['year']])

test[['month']] = scaler.fit_transform(test[['month']])
test[['year']] = scaler.fit_transform(test[['year']])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [40]:
test.fillna(0, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [41]:
X.head()

,site_code,year,month,region,district,product_code,site_type,distance
0,119,1.535631,-1.507543,11,0,5,1,-0.57689
1,119,1.535631,-1.507543,11,0,3,1,-0.57689
2,119,1.535631,-1.507543,11,0,2,1,-0.57689
3,119,1.535631,-1.507543,11,0,6,1,-0.57689
4,119,1.535631,-1.507543,11,0,7,1,-0.57689


In [42]:
test.head()

,site_code,year,month,region,district,product_code,site_type,distance
35753,119,0.0,-1.220987,11,0,1,1,-0.527911
35754,119,0.0,-1.220987,11,0,5,1,-0.527911
35755,119,0.0,-1.220987,11,0,3,1,-0.527911
35756,119,0.0,-1.220987,11,0,2,1,-0.527911
35757,119,0.0,-1.220987,11,0,6,1,-0.527911


In [43]:
y.stock_distributed.isna().sum()

0

In [44]:
y_train = y.stock_distributed.values

In [45]:
X_train = X.values
X_test = test.values

In [46]:
print(X_train.shape,X_test.shape)

(35753, 8) (3089, 8)


In [48]:
df.head()

,index,site_code,year,month,region,district,product_code,stock_distributed,train_or_test,site_type,lat,long
0,0,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27134,21.0,train,Hospital,6.730937,-3.483863
1,1,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27132,3.0,train,Hospital,6.730937,-3.483863
2,2,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27000,22.0,train,Hospital,6.730937,-3.483863
3,3,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27137,0.0,train,Hospital,6.730937,-3.483863
4,4,C4001,2019,1,INDENIE-DJUABLIN,ABENGOUROU,AS27138,2.0,train,Hospital,6.730937,-3.483863


In [49]:
X_data_train = X[['site_code'	,'year'	,'month'	,'region'	,'district'	,'product_code'	,'site_type'	,'distance']]
X_data_test = test[['site_code'	,'year'	,'month'	,'region'	,'district'	,'product_code'	,'site_type'	,'distance']]

In [ ]:
# reshaping input
#x_train = X_train.reshape(X_train.shape[0], 1,X_train.shape[1]); xtest = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

In [51]:
'Differentiable Neural Computer (Memory Augmented Neural Network)'

'Differentiable Neural Computer (Memory Augmented Neural Network)'

In [53]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random

from datetime import datetime


tf.compat.v1.set_random_seed(10)
tf.compat.v1.reset_default_graph()

random.seed(0)
np.random.seed(0)

tf.compat.v1.reset_default_graph()
tf.compat.v1.disable_eager_execution()

from tensorflow.compat.v1.nn.rnn_cell import LSTMCell, BasicRNNCell, GRUCell
from tensorflow.compat.v1.nn import static_rnn, static_bidirectional_rnn

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [54]:
num_time_steps = 1
output_dim = 1
input_features = 8
batch_size = 193
hid_dim= 128

N = 50
W = 10
R = 5

'HELPER METHODS'

def reshape_and_softmax(r_read_modes):
        r_read_modes = tf.reshape(r_read_modes, [batch_size, R, 3])
        return tf.nn.softmax(r_read_modes, axis=2)
    
def oneplus(x):
    return 1 + tf.nn.softplus(x)
    
sizes = [R * W, R, W, 1, W, W, R, 1, 1]
names = ["r_read_keys", "r_read_strengths", "write_key", "write_strength", "erase_vector", "write_vector",
         "r_free_gates", "allocation_gate", "write_gate"]
functions = [tf.identity, oneplus, tf.identity, oneplus, tf.nn.sigmoid, tf.identity,
             tf.nn.sigmoid, tf.nn.sigmoid, tf.nn.sigmoid]
indexes = [[sum(sizes[:i]), sum(sizes[:i + 1])] for i in range(len(sizes))]
assert len(names) == len(sizes) == len(functions) == len(indexes)
# Computing the interface dictionary from ordered lists of sizes, vector names and functions, as in the paper
# self.split_interface is called later with the actual computed interface vector
split_interface = lambda iv: {name: fn(iv[:, i[0]:i[1]]) for name, i, fn in
                                   zip(names, indexes, functions)}

epsilon = 1e-6
epsilon = int(epsilon)

def content_lookup(memory, keys, strength):
    """

    :param memory: array of shape [batch_size, memory_size, word_size], i.e. [b, n, w]
    :param keys: array of shape [batch_size, n_keys, word_size], i.e. [b, r, w]
    :param strength: array of shape [batch_size, n_keys], i.e. [b, r]
    :return: tensor of shape [batch_size, memory_size, n_keys]
    """
    keys = tf.nn.l2_normalize(keys, axis=2)
    memory = tf.nn.l2_normalize(memory, axis=2)
    similarity = tf.einsum("bnw,brw,br->bnr", memory, keys, strength)
    content_weighting = tf.nn.softmax(similarity, axis=1, name="Content_weighting")

    return content_weighting

def calculate_allocation_weighting(usage_vec):
    usage_vector = epsilon + (1 - epsilon) * usage_vec

    highest_usage, inverse_indices = tf.nn.top_k(-usage_vector, k=N*W)
    lowest_usage = -highest_usage

    allocation_scrambled = (1 - lowest_usage) * tf.math.cumprod(lowest_usage, axis=1, exclusive=True)

    # allocation is not in the correct order. alloation[i] contains the sorted[i] value
    # reversing the already inversed indices for each batch
    indices = tf.stack([tf.math.invert_permutation(batch_indices) for batch_indices in tf.unstack(inverse_indices)])
    allocation = tf.stack([tf.gather(mem, ind)
                           for mem, ind in
                           zip(tf.unstack(allocation_scrambled), tf.unstack(indices))])

    return allocation

In [55]:
# LSTM
epsilon = 1e-6
epsilon = int(epsilon)   
class DNC: 
    def __init__(self, x, N, W, R, batch_size):
        
        self.x = x
        print('X: ', self.x.shape)
        self.N = N
        print('N: ', N)
        self.W = W
        print('W: ', W)
        self.R = R
        print('R: ', R)
        self.batch_size = batch_size
        print('batch_size: ', batch_size)
        self.memory_size = self.N*self.W
        print('MEMORY SIZE: ', self.memory_size)

        self.memory = tf.fill([self.batch_size, self.N*self.W, self.W], 1)
        print('MEMORY: ', self.memory)
        self.memory = tf.cast(self.memory, tf.float32)

        self.read_vectors = tf.fill([self.batch_size, self.R*self.W], 1)
        self.read_vectors = tf.cast(self.read_vectors, tf.float32)
        print('READ VECTORS: ', self.read_vectors.shape)

        self.input_dim = self.read_vectors.shape[-1] + input_features 
        print('FEATURES: ', input_features) 
        print('INPUT DIM: ', self.input_dim) #CONTROLLER INPUT DIMENSION IS SUM OF INPUT FEATURES AND READ VECTOR SIZE


        self.w1 = tf.Variable(tf.compat.v1.random_normal([hid_dim, hid_dim]), dtype=tf.float32)
        self.b1 = tf.Variable(tf.compat.v1.random_normal([hid_dim]), dtype=tf.float32)

        self.w2 = tf.Variable(tf.compat.v1.random_normal([hid_dim, output_dim]), dtype=tf.float32)
        self.b2 = tf.Variable(tf.compat.v1.random_normal([output_dim]), dtype=tf.float32)

        self.interface_vec_size = (self.W*self.R) + 3*self.W + 5*self.R + 3
        self.interface_weights = tf.Variable(tf.compat.v1.random_normal([hid_dim, self.interface_vec_size]), dtype=tf.float32)
        print('INTERFACE WEIGHTS: ', self.interface_weights.shape)

        self.memory_weights = tf.Variable(tf.compat.v1.random_normal([self.R* self.W, output_dim]))
        print('MEMORY WEIGHTS: ', self.memory_weights.shape)

        #initial interface weightings
        self.write_weighting = tf.fill([self.batch_size, self.N*self.W], epsilon, name="Write_weighting")
        self.write_weighting = tf.cast(self.write_weighting, tf.float32)

        self.usage_vector = tf.zeros([self.batch_size, self.N*self.W], name="Usage_vector")

        self.read_weightings = tf.fill([self.batch_size, self.N*W, self.R], epsilon)
        self.read_weightings = tf.cast(self.read_weightings, tf.float32)
        
        self.sizes = [self.R * self.W, self.R, self.W, 1, self.W, self.W, self.R, 1, 1]
        self.names = ["r_read_keys", "r_read_strengths", "write_key", "write_strength", "erase_vector", "write_vector",
                 "r_free_gates", "allocation_gate", "write_gate"]
        self.functions = [tf.identity, oneplus, tf.identity, oneplus, tf.nn.sigmoid, tf.identity,
                     tf.nn.sigmoid, tf.nn.sigmoid, tf.nn.sigmoid]

        self.indexes = [[sum(self.sizes[:i]), sum(self.sizes[:i + 1])] for i in range(len(self.sizes))]
        assert len(self.names) == len(self.sizes) == len(self.functions) == len(self.indexes)
        self.split_interface = lambda iv: {name: fn(iv[:, i[0]:i[1]]) for name, i, fn in
                                           zip(self.names, self.indexes, self.functions)}

        
    def RNN(self, x, weights, biases):
        self.x = tf.unstack(self.x, num_time_steps, 1)
        self.cell = LSTMCell(hid_dim)
        self.outputs, self.states, = static_rnn(self.cell,  self.x, dtype=tf.float32)
        return tf.matmul(self.outputs[-1], weights) + biases
    
    def step(self): 
        #global read_vectors, usage_vector, memory, write_weighting
        self.x = tf.concat([self.x, self.read_vectors], axis=1) 
        print('X AFTER CONCATENATION: ', self.x.shape)

        self.x = tf.reshape(self.x, [batch_size, num_time_steps, self.input_dim])
        print('X RESHAPED FOR RNN: ', self.x.shape)

        self.h1 = self.RNN(self.x, self.w1, self.b1)
        print('H1: ', self.h1.shape)

        self.controller_out = tf.matmul(self.h1, self.w2) + self.b2
        print('CONTROLLER OUTPUT: ', self.controller_out.shape)

        self.interface_out = tf.matmul(self.h1, self.interface_weights)
        print('INTERFACE OUTPUT: ', self.interface_out.shape)

        self.intf = split_interface(self.interface_out)
        #reshape keys
        self.intf["write_key"] = tf.expand_dims(self.intf["write_key"], axis=1)
        self.intf["r_read_keys"] = tf.reshape(self.intf["r_read_keys"], [-1, R, W])
        #WRITE
        self.write_content = content_lookup(self.memory, self.intf['write_key'], self.intf['write_strength'])
        # dynamic allocation
        self.memory_retention = tf.reduce_prod(1 - tf.einsum("br,bnr->bnr", self.intf["r_free_gates"], self.read_weightings), 2)
        self.usage_vector = (self.usage_vector + self.write_weighting - self.usage_vector * 
                             self.write_weighting) * self.memory_retention

        self.write_allocation = calculate_allocation_weighting(self.usage_vector)
        #final write calculations
        self.write_weighting = self.intf["write_gate"] * (self.intf["allocation_gate"] * self.write_allocation
                                                      + tf.einsum("bnr,bi->bn",
                                                                  self.write_content,
                                                                  (1 - self.intf["allocation_gate"])))
        self.memory = self.memory * (1 - tf.einsum("bn,bw->bnw", self.write_weighting, self.intf["erase_vector"])) + \
                    tf.einsum("bn,bw->bnw", self.write_weighting, self.intf["write_vector"])
        #
        #READ 
        
        self.read_content_lookup = content_lookup(self.memory, self.intf['r_read_keys'], self.intf['r_read_strengths'])
        print('READ LOOK-UP WEIGHTS: ', self.read_content_lookup.shape)
        self.read_weightings = self.read_content_lookup
        print('READ WEIGHTING: ', self.read_weightings.shape)
        self.current_read_vectors = tf.einsum("bnw,bnr->brw", self.memory, self.read_weightings)
        self.current_read_vectors = tf.reshape(self.current_read_vectors, (-1, self.W * self.R))
        print('READ VECTORS: ', self.current_read_vectors.shape)
        self.memory_output = tf.matmul(self.current_read_vectors, self.memory_weights)
        print('MEMORY OUTPUT: ', self.memory_output.shape)

        self.read_vectors = self.current_read_vectors
        
        #bypass dropout
        #self.controller_out = tf.nn.dropout(self.controller_out, rate=0.1)

        self.final_out = self.controller_out + self.memory_output  
        print('FINAL OUTPUT: ', self.final_out.shape)
        print(self.final_out)
        return self.final_out


In [56]:
tf.compat.v1.reset_default_graph()

num_time_steps = 1

batch_size = 193
features = 8

input_dim_init = features



x = tf.compat.v1.placeholder(tf.float32, (batch_size, features), name='x')
print('X: ', x.shape)
y = tf.compat.v1.placeholder(tf.float32, (batch_size, ), name='y')
print('Y: ', y.shape)

a = DNC(x, N, W, R, batch_size)

out = a.step()
loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y, out))))
optimizer = tf.compat.v1.train.AdamOptimizer(0.001).minimize(loss)
#pred = tf.nn.softmax(out)
pred = out

print(pred.shape)


X:  (193, 8)
Y:  (193,)
X:  (193, 8)
N:  50
W:  10
R:  5
batch_size:  193
MEMORY SIZE:  500
MEMORY:  Tensor("Fill:0", shape=(193, 500, 10), dtype=int32)
READ VECTORS:  (193, 50)
FEATURES:  8
INPUT DIM:  58
INTERFACE WEIGHTS:  (128, 108)
MEMORY WEIGHTS:  (50, 1)
X AFTER CONCATENATION:  (193, 58)
X RESHAPED FOR RNN:  (193, 1, 58)
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
H1:  (193, 128)
CONTROLLER OUTPUT:  (193, 1)
INTERFACE OUTPUT:  (193, 108)
READ LOOK-UP WEIGHTS:  (193, 500, 5)
READ WEIGHTING:  (193, 500, 5)
READ VECTORS:  (193, 50)
MEMORY OUTPUT:  (193, 1)
FINAL OUTPUT:  (193, 1)
Tensor("add_8:0", shape=

In [57]:
y_train.shape

(35753,)

In [58]:
from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [60]:
print(X_train.shape)
ten_percent = 3575
train_in = X_train[3575:]
train_target = y_train[3575:]

val_in = X_train[:3575]
val_target = y_train[:3575]

print(train_in.shape)
print(val_in.shape)

(35753, 8)
(32178, 8)
(3575, 8)


In [ ]:
int(X_train.shape[0])/10

3575.3

In [61]:
val_target.shape

(3575,)

In [ ]:
import time

train_in = train_in
val_in = val_in

train_target = train_target
val_target = val_target
train_target = np.array(train_target)
val_target = np.array(val_target)

y_t = train_target
y_t_val = val_target

num_of_train_batches = len(train_in)/batch_size
num_of_test_batches = len(val_in)/batch_size

init=tf.compat.v1.global_variables_initializer()


print('TRAIN BATCHES: ', num_of_train_batches)
print('VAL BATCHES: ', num_of_test_batches)

train_accuracy_list = []

epoch_list = []
epoch_losses = []
epoch_test_losses = []

saver = tf.compat.v1.train.Saver()

epochs = 25
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
      lt = []
      ft = 0
      tt = 1

      lv = []
      fv = 0
      tv = 1

      train_losses = []
      test_losses = []

      predictions = []

      print('EPOCH: ', epoch)
      epoch_list.append(epoch)
      print('TRAINING')
      start = time.time()
      # RUN WHOLE SET
      for it in range(int(num_of_train_batches)): #len(x_train)/batch_size
          #print(it)
          # OPTIMIZE
          sess.run(optimizer, feed_dict={x:train_in[ft*batch_size:tt*batch_size], y:train_target[ft*batch_size:tt*batch_size]})
          #FULL PREDICT TRAIN
          p = sess.run(pred, feed_dict={x:train_in[ft*batch_size:tt*batch_size]})
          batch_loss = sess.run(root_mean_squared_error(y_true=train_target[ft*batch_size:tt*batch_size], 
                                               y_pred=p))
          #print(batch_loss)
          train_losses.append(batch_loss)
          
 
          ft+=1
          tt+=1
      # RUN WHOLE TEST SET
      for it in range(int(num_of_test_batches)): #len(x_train)/batch_size
          #FULL PREDICT TRAIN
          p = sess.run(pred, feed_dict={x:val_in[fv*batch_size:tv*batch_size]})
          predictions.append(p)
          batch_loss = sess.run(root_mean_squared_error(y_true=val_target[fv*batch_size:tv*batch_size], 
                                              y_pred=p))
          test_losses.append(batch_loss)
           
          
 
          fv+=1
          tv+=1
      
  

      # PRINT

      epoch_losses.append(np.array(train_losses).mean())
      epoch_test_losses.append(np.array(test_losses).mean())

      print('EPOCH: ', epoch)
      print('LOSS: ', np.array(train_losses).mean())
      print('TEST LOSS: ', np.array(test_losses).mean())
      #print('FULL SET ACCURACY: ', train_accuracy)
      #print('FULL VAL ACCURACY: ', val_accuracy)

      end = time.time()
      time_took = end - start

      print('TIME TOOK: ', time_took/60)

      saver.save(sess, 'model_iter', global_step=epoch)

      sns.lineplot(epoch_list, epoch_losses, color='purple', label='loss')
      sns.lineplot(epoch_list, epoch_test_losses, color='BLACK', label='loss')
      plt.show()

TRAIN BATCHES:  166.72538860103626
VAL BATCHES:  18.523316062176164
EPOCH:  0
TRAINING


In [ ]:
# SAVING AND RESTORING THE MODEL
tf.compat.v1.reset_default_graph()
imported_meta = tf.compat.v1.train.import_meta_graph("model_iter-18.meta")



In [ ]:
test_batches = X_test.shape[0]/batch_size
predictions = []
print(test_batches)
with tf.compat.v1.Session() as sess:
    imported_meta.restore(sess, tf.compat.v1.train.latest_checkpoint('./'))
    f = 0
    t = 1

    for i in range(int(test_batches)):
        prediction = sess.run('add_8:0', feed_dict={'x:0':X_test[f*batch_size:t*batch_size]})
        predictions.append(prediction)



16.00518134715026
INFO:tensorflow:Restoring parameters from ./model_iter-18


In [ ]:
np.array(predictions).shape


(16, 193, 1)

In [ ]:
fp = np.array(predictions).flatten()
fp.shape

(3088,)

In [ ]:
z = np.lib.pad(fp, (0,1))
z.shape

(3089,)

In [ ]:
z[3088] = 11
z[3088]

11.0

In [ ]:
sub = pd.read_csv('SampleSubmission.csv')
sub.shape

(3089, 2)

In [ ]:
sub['prediction'] = z

In [ ]:
sub.to_csv('d.csv', index=False)

In [ ]:
sub = pd.read_csv('SampleSubmission.csv')
sub.shape

(3089, 2)

In [ ]:
from sklearn.model_selection import KFold
init = tf.compat.v1.global_variables_initializer()

def run_train(session, train_x, train_y):

  print("\nStart training")
  session.run(init)
  train_in = train_x
  train_target = train_y
  train_target = np.array(train_target)
  y_t = train_y
  num_of_train_batches = len(train_in)/batch_size
  num_of_test_batches = len(test)/batch_size
  train_accuracy_list = []
  epoch_list = []
  epoch_losses = []
  epochs = 10
  with tf.compat.v1.Session() as sess:
      sess.run(init)
      for epoch in range(epochs):
        lt = []
        ft = 0
        tt = 1

        train_losses = []

        print('EPOCH: ', epoch)
        epoch_list.append(epoch)
        start = time.time()
        # RUN WHOLE SET
        for it in range(int(num_of_train_batches)): #len(x_train)/batch_size
            #print(it)
            # OPTIMIZE
            sess.run(optimizer, feed_dict={x:train_in[ft*batch_size:tt*batch_size], y:train_target[ft*batch_size:tt*batch_size]})
            #FULL PREDICT TRAIN
            p = sess.run(pred, feed_dict={x:train_in[ft*batch_size:tt*batch_size]})
            batch_loss = sess.run(root_mean_squared_error(y_true=train_target[ft*batch_size:tt*batch_size], 
                                                y_pred=p))
            #print(batch_loss)
            train_losses.append(batch_loss)
            ft+=1
            tt+=1

        # PRINT
        epoch_losses.append(np.array(train_losses).mean())
        print('LOSS: ', np.array(train_losses).mean())
        #print('FULL SET ACCURACY: ', train_accuracy)
        #print('FULL VAL ACCURACY: ', val_accuracy)
        end = time.time()
        time_took = end - start
        print('TIME TOOK: ', time_took/60)
        #sns.lineplot(epoch_list, epoch_losses, color='purple', label='loss')
        plt.show()

def cross_validate(session, split_size=5):
  results = []
  kf = KFold(n_splits=split_size)
  for train_idx, val_idx in kf.split(train_x_all, train_y_all):
    train_x = train_x_all[train_idx]
    train_y = train_y_all[train_idx]
    val_x = train_x_all[val_idx]
    val_y = train_y_all[val_idx]
    run_train(session, train_x, train_y)

    num_of_test_batches = len(val_x)/batch_size

    fv = 0
    tv = 1

    VAL_LOSSES = []
    for it in range(int(num_of_test_batches)):
        #FULL PREDICT VAL
        p = sess.run(pred, feed_dict={x:val_x[fv*batch_size:tv*batch_size]})
        batch_loss = sess.run(root_mean_squared_error(y_true=val_y[ft*batch_size:tt*batch_size], 
                                               y_pred=p))
        predictions.append(p)

        fv+=1
        tv+=1
    results.append(session.run(accuracy, feed_dict={x: val_x, y: val_y}))
  return results